# DS-Project-Assignment

##BITF20M025 :LAIBA MUBASHIR
##BITF20M020 :AREESHA KASHIF


In [ ]:
#Connect Google Drive

from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, recall_score
from scipy.stats import skew, kurtosis
from numpy import mean, absolute

In [ ]:
# Load Data

path_train = "/content/drive/MyDrive/DS/bbh/training/"
path_test = "/content/drive/MyDrive/DS/bbh/testing/"

trainLabels = np.load(path_train + "trainLabels.npy")
trainMSAccelerometer = np.load(path_train + "trainMSAccelerometer.npy")
trainMSGyroscope = np.load(path_train + "trainMSGyroscope.npy")
trainJinsAccelerometer = np.load(path_train + "trainJinsAccelerometer.npy")
trainJinsGyroscope = np.load(path_train + "trainJinsGyroscope.npy")
trainAccelerometer = np.load(path_train + "trainAccelerometer.npy")
trainGravity = np.load(path_train + "trainGravity.npy")
trainGyroscope = np.load(path_train + "trainGyroscope.npy")
trainLinearAcceleration = np.load(path_train + "trainLinearAcceleration.npy")
trainMagnetometer = np.load(path_train + "trainMagnetometer.npy")

testLabels = np.load(path_test + "testLabels.npy")
testMSAccelerometer = np.load(path_test + "testMSAccelerometer.npy")
testMSGyroscope = np.load(path_test + "testMSGyroscope.npy")
testJinsAccelerometer = np.load(path_test + "testJinsAccelerometer.npy")
testJinsGyroscope = np.load(path_test + "testJinsGyroscope.npy")
testAccelerometer = np.load(path_test + "testAccelerometer.npy")
testGravity = np.load(path_test + "testGravity.npy")
testGyroscope = np.load(path_test + "testGyroscope.npy")
testLinearAcceleration = np.load(path_test + "testLinearAcceleration.npy")
testMagnetometer = np.load(path_test + "testMagnetometer.npy")

#print dimensions of all loaded nparrays

print(trainMSAccelerometer.shape, trainMSGyroscope.shape, trainJinsAccelerometer.shape, trainJinsGyroscope.shape, trainAccelerometer.shape,
      trainGravity.shape, trainGyroscope.shape, trainLinearAcceleration.shape, trainMagnetometer.shape, trainLabels.shape)

print(testMSAccelerometer.shape, testMSGyroscope.shape, testJinsAccelerometer.shape, testJinsGyroscope.shape, testAccelerometer.shape,
      testGravity.shape, testGyroscope.shape, testLinearAcceleration.shape, testMagnetometer.shape, testLabels.shape)

(2284, 268, 3) (2284, 268, 3) (2284, 80, 3) (2284, 80, 3) (2284, 800, 3) (2284, 800, 3) (2284, 800, 3) (2284, 800, 3) (2284, 200, 3) (2284,)
(2288, 268, 3) (2288, 268, 3) (2288, 80, 3) (2288, 80, 3) (2288, 800, 3) (2288, 800, 3) (2288, 800, 3) (2288, 800, 3) (2288, 200, 3) (2288,)


In [ ]:
print(trainLabels)
print(testLabels)

[47 47 22 ... 33 29 28]
[17 10 34 ... 29 34 10]


In [ ]:
#create a list of file names
test_list=[testAccelerometer,testGravity,testGyroscope,testJinsAccelerometer,
      testJinsGyroscope,testLinearAcceleration,testMagnetometer,
      testMSAccelerometer,testMSAccelerometer]

train_list=[trainAccelerometer,trainGravity,trainGyroscope,trainJinsAccelerometer,
      trainJinsGyroscope,trainLinearAcceleration,trainMagnetometer,
      trainMSAccelerometer,trainMSAccelerometer]

In [ ]:
#Applying Normalization
def Apply_Normalization(input_arr):
  for i in range(0,3):
    input_arr[:,:,i]=input_arr[:,:,i]/np.linalg.norm(input_arr[:,:,i])
  return input_arr

#normalize train dataset
for file in train_list:
  file=Apply_Normalization(file)
#normalize test dataset
for file in test_list:
  file=Apply_Normalization(file)

In [ ]:
print(trainMSAccelerometer.shape, trainMSGyroscope.shape, trainJinsAccelerometer.shape, trainJinsGyroscope.shape, trainAccelerometer.shape,
      trainGravity.shape, trainGyroscope.shape, trainLinearAcceleration.shape, trainMagnetometer.shape)

print(testMSAccelerometer.shape, testMSGyroscope.shape, testJinsAccelerometer.shape, testJinsGyroscope.shape, testAccelerometer.shape,
      testGravity.shape, testGyroscope.shape, testLinearAcceleration.shape, testMagnetometer.shape)

(2284, 268, 3) (2284, 268, 3) (2284, 80, 3) (2284, 80, 3) (2284, 800, 3) (2284, 800, 3) (2284, 800, 3) (2284, 800, 3) (2284, 200, 3)
(2288, 268, 3) (2288, 268, 3) (2288, 80, 3) (2288, 80, 3) (2288, 800, 3) (2288, 800, 3) (2288, 800, 3) (2288, 800, 3) (2288, 200, 3)


In [ ]:
#create a new array that will contain extracted features
#feaure extraction (mean, max, min, std, var, median, percentile 20, percentile 50, percentile 80, kurtosis, skewness, zero crossing, first mean, second mean)
num_features = 14
train_features = np.zeros((np.shape(trainLabels)[0], 9*num_features, 3))
test_features = np.zeros((np.shape(testLabels)[0], 9*num_features, 3))

print(np.shape(train_features))
print(np.shape(test_features))

(2284, 126, 3)
(2288, 126, 3)


In [ ]:


def feature_extract(features, files, feature_index):
    for file in files:
        features[:, feature_index, :] = np.mean(file, axis=1)
        features[:, feature_index + 1, :] = np.max(file, axis=1)
        features[:, feature_index + 2, :] = np.min(file, axis=1)
        features[:, feature_index + 3, :] = np.std(file, axis=1)
        features[:, feature_index + 4, :] = np.var(file, axis=1)
        features[:, feature_index + 5, :] = np.median(file, axis=1)
        features[:, feature_index + 6, :] = np.percentile(file, 20, axis=1)
        features[:, feature_index + 7, :] = np.percentile(file, 50, axis=1)
        features[:, feature_index + 8, :] = np.percentile(file, 80, axis=1)
        features[:, feature_index + 9, :] = kurtosis(file, axis=1)
        features[:, feature_index + 10, :] = skew(file, axis=1)
        features[:, feature_index + 11, :] = np.sum(np.diff(np.sign(file), axis=1) != 0, axis=1)
        features[:, feature_index + 12, :] = np.mean(file, axis=1) #frist mean
        features[:, feature_index + 13, :] = np.mean(file, axis=1)**2 #second mean
        feature_index += 14

feature_extract(train_features, train_list, 0)
feature_extract(test_features, test_list, 0)

print(np.shape(train_features))
print(np.shape(test_features))

print(train_features)
print(test_features)


<ipython-input-8-e96306533ccc>:12: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features[:, feature_index + 9, :] = kurtosis(file, axis=1)
<ipython-input-8-e96306533ccc>:13: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features[:, feature_index + 10, :] = skew(file, axis=1)


(2284, 126, 3)
(2288, 126, 3)
[[[-8.56235056e-05  9.52838571e-04 -1.34265079e-04]
  [ 1.11980026e-03  1.57776254e-03  6.46544679e-04]
  [-1.32178178e-03  5.78874140e-04 -9.47633176e-04]
  ...
  [ 0.00000000e+00  0.00000000e+00  1.10000000e+01]
  [ 1.91500934e-03  5.96664788e-04  1.01045909e-04]
  [ 3.66726067e-06  3.56008883e-07  1.02102762e-08]]

 [[ 7.00020828e-05  9.55460826e-04 -1.33648922e-04]
  [ 5.66039351e-04  1.37143978e-03  2.80281529e-04]
  [-8.74843914e-04  7.65908393e-04 -6.33693358e-04]
  ...
  [ 0.00000000e+00  4.00000000e+00  1.70000000e+01]
  [ 1.91298465e-03  6.42751926e-04 -1.27375490e-04]
  [ 3.65951018e-06  4.13130039e-07  1.62245151e-08]]

 [[-3.17084850e-05  2.10744503e-04 -1.23783271e-03]
  [ 9.45445354e-05  2.48367578e-04 -1.19609234e-03]
  [-2.27152457e-04  1.81832802e-04 -1.26952853e-03]
  ...
  [ 1.00000000e+01  2.20000000e+01  0.00000000e+00]
  [ 3.67557834e-04  8.24879564e-04  1.77300000e-03]
  [ 1.35098759e-07  6.80426297e-07  3.14352906e-06]]

 ...

 [[-

In [ ]:
#Reshape the original numpy arrays
train_features_reshaped = np.reshape(train_features, (np.shape(train_features)[0], np.shape(train_features)[1]*np.shape(train_features)[2] ))
print(np.shape(train_features_reshaped))


test_features_reshaped = np.reshape(test_features, (np.shape(test_features)[0], np.shape(test_features)[1]*np.shape(test_features)[2] ))
print(np.shape(test_features_reshaped))

print(train_features_reshaped)
print(test_features_reshaped)

(2284, 378)
(2288, 378)
[[-8.56235056e-05  9.52838571e-04 -1.34265079e-04 ...  3.66726067e-06
   3.56008883e-07  1.02102762e-08]
 [ 7.00020828e-05  9.55460826e-04 -1.33648922e-04 ...  3.65951018e-06
   4.13130039e-07  1.62245151e-08]
 [-3.17084850e-05  2.10744503e-04 -1.23783271e-03 ...  1.35098759e-07
   6.80426297e-07  3.14352906e-06]
 ...
 [-1.64224373e-04  9.31430375e-04 -2.91434553e-04 ...  3.76312028e-06
   1.76778883e-07  3.64976316e-09]
 [-1.83276599e-03  6.74374343e-04  1.64868252e-05 ...  3.69642663e-07
   2.30363412e-07  4.68138614e-06]
 [-8.88742215e-05  1.74346700e-04 -1.24468585e-03 ...  4.03991294e-07
   3.06070376e-07  4.48010724e-06]]
[[ 1.49244932e-03  1.88917591e-04 -4.26642713e-04 ...  3.85865988e-08
   3.47205628e-07  6.18037006e-07]
 [ 2.12274084e-04  1.69338862e-04 -1.23894820e-03 ...  3.73414395e-07
   2.18377545e-06  7.40725795e-07]
 [ 3.71902279e-04  2.19639580e-04 -1.22356834e-03 ...  1.34102400e-08
   2.55178429e-06  1.58431226e-06]
 ...
 [-1.66977174e-04  9

In [ ]:

# Replace null values with the minimum of the data
min_value = np.nanmin(train_features_reshaped)
train_features_reshaped[np.isnan(train_features_reshaped)] = min_value

min_value = np.nanmin(test_features_reshaped)
test_features_reshaped[np.isnan(test_features_reshaped)] = min_value

# Create an instance of SVC
classifier = SVC(kernel='linear', C=1.0)

# Fit the model to the imputed training data
classifier.fit(train_features_reshaped, trainLabels)

# Make predictions on the imputed test data
estimatedLabels = classifier.predict(test_features_reshaped)

accuracy = accuracy_score(testLabels, estimatedLabels)
recall = recall_score(testLabels, estimatedLabels, average='macro')
weightedF1 = f1_score(testLabels, estimatedLabels, average='weighted')
averageF1 = f1_score(testLabels, estimatedLabels, average='macro')
conf_matrix = confusion_matrix(testLabels, estimatedLabels)

# Print results
print('   Average F1-score = %.4f' % (averageF1))
print('   Test Recall = %.4f' % (recall * 100))
print('   Test accuracy = %.2f %%' % (accuracy * 100))
print('   Weighted F1-score = %.4f' % (weightedF1))
print('   Confusion Matrix:')
print(conf_matrix)

print('-------------------------------------------------------')


   Average F1-score = 0.3312
   Test Recall = 33.6829
   Test accuracy = 33.48 %
   Weighted F1-score = 0.3309
   Confusion Matrix:
[[36  0  0 ...  0  0  0]
 [ 0 35  5 ...  0  0  0]
 [ 1  4 19 ...  1  3  0]
 ...
 [ 0  1  2 ...  7  0  1]
 [ 0  0  1 ...  0 11  0]
 [ 0  0  0 ...  0  0 12]]
-------------------------------------------------------
